# Douyin Live Recorder

Notebook นี้ใช้สำหรับรัน DouyinLiveRecorder เพื่อบันทึกวิดีโอไลฟ์จาก Douyin (TikTok จีน)

In [ ]:
# Clone repositories
!git clone https://github.com/ihmily/DouyinLiveRecorder.git
!git clone https://github.com/chaiz64/l777k.git

In [ ]:
# Copy configuration files
!cp l777k/config.ini DouyinLiveRecorder/config/
!cp l777k/URL_config.ini DouyinLiveRecorder/config/

In [ ]:
# เข้าไปยังโฟลเดอร์โปรเจค
%cd DouyinLiveRecorder

# ติดตั้ง dependencies
!pip3 install -r requirements.txt

In [ ]:
# ติดตั้ง ffmpeg
!apt update
!apt install ffmpeg -y
!pip install --upgrade --no-cache-dir pandas pyarrow

In [ ]:
# รันโปรแกรม
!python main.py

In [ ]:
import time
import datetime

def keep_colab_alive(duration_hours=24):
    """
    Keeps the Google Colab runtime active by running a loop.

    Args:
        duration_hours (int): The duration in hours for which the script
                              should attempt to keep the runtime alive.
                              Default is 24 hours. Set to 0 for indefinite run
                              (requires manual stop).
    """
    print(f"Script will attempt to keep Colab alive for {duration_hours} hours. (Set to 0 for indefinite run)")
    start_time = datetime.datetime.now()
    end_time = start_time + datetime.timedelta(hours=duration_hours)

    counter = 0
    try:
        while True:
            current_time = datetime.datetime.now()
            if duration_hours > 0 and current_time >= end_time:
                print(f"Maximum duration of {duration_hours} hours reached. Stopping script.")
                break

            counter += 1
            print(f"[{current_time.strftime('%Y-%m-%d %H:%M:%S')}] Colab is alive! Iteration: {counter}")
            # Sleep for a short period to prevent excessive CPU usage
            time.sleep(60) # Sleep for 60 seconds (1 minute)

    except KeyboardInterrupt:
        print("\nScript stopped manually by user (Ctrl+C).")
    except Exception as e:
        print(f"\nAn error occurred: {e}")
    finally:
        print("Colab keep-alive script finished.")

# --- How to use ---
# Call the function to start keeping Colab alive.
# You can specify the duration in hours.
# For example, to keep it alive for 12 hours:
# keep_colab_alive(duration_hours=12)

# To run indefinitely until manually stopped (Ctrl+C):
keep_colab_alive(duration_hours=0) # Set to 0 for indefinite run


In [ ]:
# @title 🚀 Elite Douyin/TikTok Live Downloader (v1.1 - Deep Scan Fixed)
import os
import sys
import json
import time
import shutil
import subprocess
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import google.colab.files

# --- 🎨 CSS & AESTHETICS ---
def inject_css():
    style = """
    <style>
        :root {
            --glass-bg: rgba(16, 18, 27, 0.95);
            --glass-border: rgba(255, 255, 255, 0.1);
            --neon-blue: #00f2ff;
            --neon-purple: #bc13fe;
            --text-main: #ffffff;
            --text-sub: #a0a0a0;
        }
        .elite-container {
            background: var(--glass-bg);
            border: 1px solid var(--glass-border);
            border-radius: 16px;
            padding: 20px;
            box-shadow: 0 8px 32px 0 rgba(0, 0, 0, 0.37);
            margin-bottom: 20px;
        }
        .jupyter-button {
            background: linear-gradient(135deg, var(--neon-blue) 0%, #00a8ff 100%) !important;
            border: none !important; border-radius: 8px !important; color: #000 !important;
            font-weight: 700 !important; transition: all 0.3s ease !important;
        }
        .jupyter-button:hover { transform: translateY(-2px); box-shadow: 0 0 15px var(--neon-blue); }
        .jupyter-button.mod-danger { background: linear-gradient(135deg, #ff4757 0%, #ff6b81 100%) !important; color: white !important; }
        .jupyter-button.mod-success { background: linear-gradient(135deg, #2ecc71 0%, #26de81 100%) !important; }
        input[type="text"] {
            background: rgba(255, 255, 255, 0.05) !important;
            border: 1px solid var(--glass-border) !important;
            color: var(--text-main) !important;
            border-radius: 6px !important; padding: 8px !important;
        }
        h2.elite-title {
            background: -webkit-linear-gradient(var(--neon-blue), var(--neon-purple));
            -webkit-background-clip: text; -webkit-text-fill-color: transparent;
            font-family: 'Roboto', sans-serif; font-weight: 900; margin-bottom: 10px;
        }
        /* Scrollbars */
        ::-webkit-scrollbar { width: 8px; }
        ::-webkit-scrollbar-track { background: transparent; }
        ::-webkit-scrollbar-thumb { background: var(--glass-border); border-radius: 4px; }
    </style>
    """
    display(HTML(style))

# --- 🔧 CONFIG & UTILS ---
class Config:
    DEFAULT_DIR = "/content/DouyinLiveRecorder/downloads/抖音直播/"
    BATCH_LIMIT = 3
    MIN_DURATION_SEC = 600 # 10 Minutes
    EXTENSIONS = ('.mp4', '.avi', '.mkv', '.mov', '.wmv', '.flv', '.webm', '.ts')
    HISTORY_FILE = "download_history.json"

    @staticmethod
    def save_history(file_path):
        history = Config.load_history()
        history.add(file_path)
        with open(Config.HISTORY_FILE, 'w') as f:
            json.dump(list(history), f)

    @staticmethod
    def load_history():
        if os.path.exists(Config.HISTORY_FILE):
            try:
                with open(Config.HISTORY_FILE, 'r') as f:
                    return set(json.load(f))
            except: return set()
        return set()

class SystemUtils:
    @staticmethod
    def install_ffmpeg():
        if not shutil.which('ffprobe'):
            print("⚙️ Installing FFmpeg...")
            subprocess.run(['apt-get', 'install', '-y', 'ffmpeg'], stdout=subprocess.DEVNULL)
            print("✅ FFmpeg Installed.")

    @staticmethod
    def get_duration(file_path):
        duration = 0.0
        ffprobe_cmd = shutil.which('ffprobe')
        if ffprobe_cmd:
            try:
                cmd = [ffprobe_cmd, '-v', 'error', '-show_entries', 'format=duration', '-of', 'json', file_path]
                result = subprocess.run(cmd, capture_output=True, text=True, timeout=10)
                data = json.loads(result.stdout)
                duration = float(data['format']['duration'])
            except: duration = SystemUtils._simulate_duration(file_path)
        else: duration = SystemUtils._simulate_duration(file_path)
        return duration, SystemUtils.format_time(duration)

    @staticmethod
    def _simulate_duration(file_path):
        try: return os.path.getsize(file_path) / (1024 * 1024 * 0.2)
        except: return 0.0

    @staticmethod
    def format_time(seconds):
        m, s = divmod(seconds, 60); h, m = divmod(m, 60)
        return f"{int(h):02d}h {int(m):02d}m {int(s):02d}s"

    @staticmethod
    def format_size(size):
        for unit in ['B', 'KB', 'MB', 'GB']:
            if size < 1024: return f"{size:.2f} {unit}"
            size /= 1024
        return f"{size:.2f} TB"

# --- 🖥️ APP CLASS ---
class EliteDownloaderApp:
    def __init__(self):
        inject_css()
        SystemUtils.install_ffmpeg()
        self.download_dir = Config.DEFAULT_DIR
        self.files_data = []; self.checkbox_map = {}; self.history = Config.load_history()

        # UI Construction
        self.log_output = widgets.Output(layout=widgets.Layout(height='150px', overflow_y='scroll', border='1px solid #333', padding='10px'))
        self.file_list_box = widgets.VBox([])

        # Header
        self.header = widgets.VBox([
            widgets.HTML('<h2 class="elite-title">🚀 Elite Downloader v1.1</h2>'),
            widgets.HTML(f"<span style='color:#777'>System: {'✅ FFprobe Ready' if shutil.which('ffprobe') else '⚠️ Simulating Duration'}</span>")
        ])

        # Controls
        self.path_input = widgets.Text(value=self.download_dir, description='📂 Path:', placeholder='/content/...')
        self.path_input.observe(lambda c: setattr(self, 'download_dir', c['new']), names='value')

        self.btn_scan = widgets.Button(description="🔄 Deep Scan", icon='search', layout=widgets.Layout(width='auto'))
        self.btn_scan.add_class('jupyter-button')
        self.btn_scan.on_click(lambda b: self._refresh_file_list())

        self.btn_dl = widgets.Button(description="⬇️ Download Batch", icon='download', layout=widgets.Layout(width='auto'))
        self.btn_dl.add_class('jupyter-button'); self.btn_dl.add_class('mod-success')
        self.btn_dl.on_click(self._on_batch_download)

        self.btn_reset = widgets.Button(description="🗑️ Reset History", icon='trash')
        self.btn_reset.add_class('jupyter-button'); self.btn_reset.add_class('mod-danger')
        self.btn_reset.on_click(self._clear_history)

        self.controls = widgets.VBox([
            widgets.HTML("<div class='elite-container'>"),
            self.path_input,
            widgets.HBox([self.btn_scan, self.btn_dl, self.btn_reset], layout=widgets.Layout(margin='10px 0')),
            widgets.HTML("</div>")
        ])

        self.main_layout = widgets.VBox([self.header, self.controls, widgets.HTML("<h3>📄 Video List</h3>"), self.file_list_box, widgets.HTML("<hr>"), self.log_output])

        # Auto-start scan
        self._refresh_file_list()

    def _log(self, msg, level="INFO"):
        color = {"INFO": "white", "SUCCESS": "#00ff9d", "ERROR": "#ff4757", "WARN": "orange"}.get(level, "white")
        with self.log_output: print(f"[{time.strftime('%H:%M:%S')}] {msg}")

    def _refresh_file_list(self):
        self._log(f"🕵️ Deep scanning: {self.download_dir}...", "INFO")
        self.files_data = []
        if not os.path.exists(self.download_dir):
            self._log(f"Path not found: {self.download_dir}", "ERROR"); return

        found_files = []
        # RECURSIVE SEARCH FIX
        for root, _, files in os.walk(self.download_dir):
            for file in files:
                if file.lower().endswith(Config.EXTENSIONS):
                    found_files.append(os.path.join(root, file))

        if not found_files:
            self._log("No video files found in tree.", "WARN");
            self.file_list_box.children = [widgets.HTML("📂 Folder tree is empty.")]; return

        found_files.sort()
        self._log(f"Found {len(found_files)} potential files. Analyzing...", "INFO")

        items = []
        self.checkbox_map = {}

        with self.log_output: # Capture stdout from any subprocess noise
            for full_path in found_files:
                d, fd = SystemUtils.get_duration(full_path)
                if d < Config.MIN_DURATION_SEC: continue

                f_info = {
                    "name": os.path.basename(full_path),
                    "path": full_path,
                    "rel_path": full_path.replace(self.download_dir, '...'),
                    "size": SystemUtils.format_size(os.path.getsize(full_path)),
                    "duration": fd
                }

                # Create UI Element
                is_done = f_info['path'] in self.history
                chk = widgets.Checkbox(value=not is_done, layout=widgets.Layout(width='30px'), indent=False)
                self.checkbox_map[chk] = f_info

                btn = widgets.Button(description="⬇️", layout=widgets.Layout(width='40px'))
                btn.on_click(lambda b, f=f_info: self._dl_single(f))

                row = widgets.HBox([
                    chk,
                    widgets.HTML(f"<div style='margin-left:10px'><b style='color:#00f2ff'>{f_info['name']}</b><br><span style='font-size:0.8em;color:#888'>{f_info['rel_path']} | {f_info['size']} | {f_info['duration']}</span></div>"),
                    btn
                ], layout=widgets.Layout(border_bottom='1px solid #333', padding='4px'))
                items.append(row)

        self.file_list_box.children = items if items else [widgets.HTML("⚠️ All files are shorter than 10 mins.")]
        if items: self._log(f"✅ Listed {len(items)} valid videos.", "SUCCESS")

    def _dl_single(self, f):
        self._log(f"Starting: {f['name']}", "INFO")
        try: google.colab.files.download(f['path']); self.history.add(f['path']); Config.save_history(f['path'])
        except Exception as e: self._log(f"Error: {e}", "ERROR")

    def _on_batch_download(self, b):
        b.disabled = True
        selected = [self.checkbox_map[c] for c in self.checkbox_map if c.value and self.checkbox_map[c]['path'] not in self.history]
        self._log(f"Batch processing {min(len(selected), Config.BATCH_LIMIT)} files...", "INFO")

        for i, f in enumerate(selected[:Config.BATCH_LIMIT]):
            self._log(f"({i+1}) Downloading: {f['name']}", "INFO")
            try:
                google.colab.files.download(f['path'])
                self.history.add(f['path']); Config.save_history(f['path'])
                time.sleep(2)
            except Exception as e: self._log(f"Fail: {e}", "ERROR")
        b.disabled = False

    def _clear_history(self, b):
        if os.path.exists(Config.HISTORY_FILE): os.remove(Config.HISTORY_FILE)
        self.history = set(); self._refresh_file_list(); self._log("History cleared.", "WARN")

# --- 🚀 RUN ---
app = EliteDownloaderApp()
display(app.main_layout)

In [ ]:
# @title 🛡️ Ultimate Elite Framework v2.5 (Anti-Ghost Protection)
# @markdown **Run this cell** to start.
# @markdown <br>🔰 **Upgrade:** Added "Anti-Ghost" logic to prevent local file traps.
# @markdown <br>⚡ **Rule:** Videos shorter than 10 minutes are hidden.

import os
import sys
import shutil
import subprocess
import logging
import hashlib
from datetime import timedelta
from dataclasses import dataclass, field
from typing import List, Optional, Tuple
from pathlib import Path

# --- 1. Dependency Check & Installation ---
def install_dependencies():
    """Installs required third-party libraries."""
    try:
        import rich
    except ImportError:
        print("✨ Installing UI dependencies...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "rich"])

install_dependencies()

from google.colab import drive
from rich.console import Console
from rich.table import Table
from rich.progress import Progress, SpinnerColumn, BarColumn, TextColumn, TimeRemainingColumn, FileSizeColumn
from rich.panel import Panel
from rich.logging import RichHandler
from rich.prompt import Prompt, Confirm
from rich import box
from rich.theme import Theme
from rich.style import Style

# --- 2. Configuration & Theming (NEON MODE) ---

custom_theme = Theme({
    "info": "bold cyan",
    "warning": "bold yellow",
    "danger": "bold bright_red",
    "success": "bold spring_green1",
    "header": "bold white",
    "key": "bold turquoise2",
    "value": "bold orchid1",
    "panel.border": "spring_green3",
})

@dataclass
class AppConfig:
    """Centralized Configuration."""
    source_dir: Path = Path('/content/DouyinLiveRecorder/downloads/抖音直播')
    target_dir: Path = Path('/content/drive/MyDrive/8888/ColabDL')
    min_duration_sec: int = 600
    enable_checksum: bool = True
    log_file: str = 'backup_log.log'
    default_op: str = 'move'

@dataclass
class VideoFile:
    index: int
    path: Path
    name: str
    size_mb: float
    duration_sec: float
    duration_str: str

# --- 3. Utilities ---

class LoggerSetup:
    @staticmethod
    def setup(log_path: Path, console: Console):
        for handler in logging.root.handlers[:]:
            logging.root.removeHandler(handler)

        logging.basicConfig(
            level="INFO",
            format="%(message)s",
            datefmt="[%X]",
            handlers=[
                RichHandler(console=console, rich_tracebacks=True, show_path=False, markup=True),
                logging.FileHandler(log_path)
            ]
        )
        return logging.getLogger("rich")

class SystemUtils:
    @staticmethod
    def install_ffmpeg(logger):
        try:
            subprocess.run(['ffprobe', '-h'], check=True, capture_output=True)
        except (subprocess.CalledProcessError, FileNotFoundError):
            logger.warning("[warning]FFmpeg not found. Installing...[/]")
            try:
                subprocess.run(['apt-get', 'update'], check=True, capture_output=True)
                subprocess.run(['apt-get', 'install', '-y', 'ffmpeg'], check=True, capture_output=True)
                logger.info("[success]FFmpeg installed successfully.[/]")
            except Exception as e:
                logger.error(f"[danger]Failed to install FFmpeg: {e}[/]")

    @staticmethod
    def calculate_md5(file_path: Path, chunk_size=8192) -> Optional[str]:
        try:
            hasher = hashlib.md5()
            with open(file_path, 'rb') as f:
                for chunk in iter(lambda: f.read(chunk_size), b''):
                    hasher.update(chunk)
            return hasher.hexdigest()
        except Exception:
            return None

    @staticmethod
    def get_duration(file_path: Path) -> float:
        try:
            cmd = ['ffprobe', '-v', 'error', '-show_entries', 'format=duration', '-of', 'default=noprint_wrappers=1:nokey=1', str(file_path)]
            result = subprocess.run(cmd, check=True, capture_output=True, text=True)
            val = result.stdout.strip()
            return float(val) if val else 0.0
        except Exception:
            return 0.0

    @staticmethod
    def format_seconds(seconds: float) -> str:
        return str(timedelta(seconds=int(seconds)))

# --- 4. Main Application ---

class BackupApp:
    def __init__(self):
        self.console = Console(theme=custom_theme)
        self.config = AppConfig()

        # Initial Logging setup (Temporary until drive mounts)
        self.logger = LoggerSetup.setup(Path('/content/backup_log.log'), self.console)
        self.video_files: List[VideoFile] = []

    def mount_drive(self):
        """
        🛡️ ANTI-GHOST MOUNTING PROTOCOL
        Ensures a clean, real connection to Google Drive.
        """
        mount_point = Path('/content/drive')

        # 1. Check for "Ghost Folder" (Exists but not a mount point)
        if mount_point.exists() and not os.path.ismount(mount_point):
            self.console.print("[warning]⚠️ Detect Ghost Folder! Cleaning up to allow real connection...[/]")
            try:
                # Remove the fake folder blocking the mount
                shutil.rmtree(mount_point)
                mount_point.mkdir()
            except Exception as e:
                self.logger.error(f"Failed to clear ghost folder: {e}")

        # 2. Perform Mount
        if not os.path.ismount(mount_point):
            self.console.print(Panel("🌥️ Connecting to Google Drive...", style="panel.border"))
            try:
                drive.mount('/content/drive', force_remount=True)
            except Exception as e:
                self.console.print(f"[danger]❌ Mount Failed: {e}[/]")
                return

        # 3. Verify Connection
        if os.path.ismount(mount_point):
            self.logger.info("[info]✅ Google Drive connected securely.[/]")
            # Re-init logger to save to Drive now that it's safe
            self.config.target_dir.mkdir(parents=True, exist_ok=True)
            self.logger = LoggerSetup.setup(self.config.target_dir / self.config.log_file, self.console)
        else:
            self.console.print("[danger]🚨 CRITICAL: Drive is NOT mounted. Operations aborted.[/]")

    def scan_files(self):
        if not self.config.source_dir.exists():
            self.console.print(Panel(
                f"[danger]❌ Source Directory Not Found[/]\nPath: [bold white]{self.config.source_dir}[/]\n\n"
                "[white]Please verify your download path.[/]",
                title="Configuration Error", style="danger"
            ))
            return

        all_files = []
        stats = {'found': 0, 'filtered_out': 0}

        with Progress(
            SpinnerColumn(style="bold spring_green1"),
            TextColumn("[bold cyan]{task.description}"),
            transient=True,
            console=self.console
        ) as progress:
            progress.add_task(f"Scanning (Ignoring < {self.config.min_duration_sec/60} mins)...", total=None)

            idx_counter = 0
            for file_path in self.config.source_dir.rglob('*'):
                if file_path.is_file():
                    if file_path.suffix.lower() not in ['.mp4', '.ts']:
                        continue

                    dur = SystemUtils.get_duration(file_path)
                    if dur < self.config.min_duration_sec:
                        stats['filtered_out'] += 1
                        continue

                    size_mb = file_path.stat().st_size / (1024 * 1024)
                    all_files.append(VideoFile(idx_counter, file_path, file_path.name, size_mb, dur, SystemUtils.format_seconds(dur)))
                    idx_counter += 1

        self.video_files = all_files

        if not all_files:
            msg = f"No videos longer than {self.config.min_duration_sec/60} minutes found."
            if stats['filtered_out'] > 0:
                msg += f"\n(Hidden {stats['filtered_out']} short files)"

            self.console.print(Panel(
                f"📂 Path: [bold white]{self.config.source_dir}[/]\n{msg}",
                title="⚠️ No Eligible Files", style="warning"
            ))
        else:
            self.logger.info(f"[success]Found {len(self.video_files)} eligible video files (> 10 mins).[/]")

    def display_files(self):
        if not self.video_files: return False

        table = Table(
            title=f"🎬 Eligible Videos (>10 mins) ({len(self.video_files)})",
            box=box.ROUNDED,
            header_style="header",
            border_style="spring_green3",
            show_lines=True
        )
        table.add_column("ID", style="key", justify="center")
        table.add_column("File Name", style="bold white")
        table.add_column("Size (MB)", style="success", justify="right")
        table.add_column("Duration", style="value", justify="center")

        for vid in self.video_files:
            table.add_row(str(vid.index), vid.name, f"{vid.size_mb:.2f}", vid.duration_str)

        self.console.print(table)
        return True

    def process_files(self):
        if not self.display_files(): return

        # 🔒 FINAL SAFETY CHECK BEFORE ACTION
        if not os.path.ismount('/content/drive'):
            self.console.print("[danger]🚨 Drive disconnected! Attempting to reconnect...[/]")
            self.mount_drive()
            if not os.path.ismount('/content/drive'):
                self.console.print("[danger]❌ Aborting: Cannot establish connection to Drive.[/]")
                return

        self.console.print("\n[bold white]⚙️ Action Required[/]")
        op_mode = Prompt.ask("Select Operation", choices=["copy", "move"], default=self.config.default_op, console=self.console)
        selection = Prompt.ask("Select Index ([spring_green1]all[/] or 1,2,3)", console=self.console)

        selected_videos = self.video_files if selection.lower() == 'all' else []
        if selection.lower() != 'all':
            try:
                indices = [int(x.strip()) for x in selection.split(',')]
                selected_videos = [v for v in self.video_files if v.index in indices]
            except:
                self.console.print("[danger]Invalid input![/]")
                return

        if not selected_videos: return

        stats = {'success': 0, 'fail': 0, 'skip': 0, 'mismatch': 0}

        with Progress(
            SpinnerColumn(style="bold spring_green1"),
            BarColumn(bar_width=None, style="grey23", complete_style="bold spring_green1"),
            TextColumn("[bold white]{task.description}"),
            FileSizeColumn(),
            console=self.console
        ) as progress:

            task_id = progress.add_task(f"[bold]{op_mode.capitalize()}ing...[/]", total=len(selected_videos))

            for vid in selected_videos:
                dst_path = self.config.target_dir / vid.name

                if dst_path.exists():
                    self.logger.info(f"[grey50]⏭️ Skipped {vid.name} (Exists)[/]")
                    stats['skip'] += 1
                    progress.advance(task_id)
                    continue

                try:
                    src_hash = SystemUtils.calculate_md5(vid.path) if self.config.enable_checksum else None

                    progress.update(task_id, description=f"{op_mode.capitalize()}: [cyan]{vid.name}[/]")
                    if op_mode == 'copy': shutil.copy2(vid.path, dst_path)
                    else: shutil.move(str(vid.path), str(dst_path))

                    if self.config.enable_checksum:
                        progress.update(task_id, description=f"Verifying: [cyan]{vid.name}[/]")
                        if src_hash == SystemUtils.calculate_md5(dst_path):
                            self.logger.info(f"[success]✔ Verified: {vid.name}[/]")
                            stats['success'] += 1
                        else:
                            self.logger.error(f"[danger]❌ Mismatch: {vid.name}[/]")
                            stats['mismatch'] += 1
                    else:
                        stats['success'] += 1

                except Exception as e:
                    self.logger.error(f"[danger]Failed {vid.name}: {e}[/]")
                    stats['fail'] += 1

                progress.advance(task_id)

        if op_mode == 'copy' and stats['success'] > 0:
            if Confirm.ask("🗑️ Delete source files?", console=self.console):
                self.delete_sources(selected_videos, self.config.target_dir)

        self.print_summary(stats, op_mode)

    def delete_sources(self, videos, target_dir):
        deleted = 0
        for vid in videos:
            if (target_dir / vid.name).exists():
                os.remove(vid.path)
                deleted += 1
        self.console.print(f"[success]🗑️ Cleaned up {deleted} files.[/]")

    def print_summary(self, stats, op_mode):
        grid = Table.grid(expand=True, padding=(0, 2))
        grid.add_column()
        grid.add_column(justify="right")

        grid.add_row(f"[bold cyan]Successfully {op_mode}ed[/]", f"[bold spring_green1]{stats['success']}[/]")
        if stats['fail']: grid.add_row("[bold red]Failed[/]", f"[bold red]{stats['fail']}[/]")
        if stats['skip']: grid.add_row("[grey50]Skipped[/]", f"[grey50]{stats['skip']}[/]")

        self.console.print(Panel(grid, title="📊 Summary", border_style="spring_green3", expand=False))
        self.console.print("[bold spring_green1]✨ All tasks completed successfully![/]")

    def run(self):
        self.console.print(Panel.fit("🛡️ [bold spring_green1]Google Drive Backup Manager[/] [bold white]Anti-Ghost Edition[/]", border_style="spring_green3"))
        SystemUtils.install_ffmpeg(self.logger)
        self.mount_drive()
        self.scan_files()
        self.process_files()
        try: drive.flush_and_unmount(); self.console.print("[grey50 italic]Drive unmounted.[/]")
        except: pass

if __name__ == "__main__":
    app = BackupApp()
    app.run()

In [ ]:
# @title 🚑 Emergency Recovery: กู้คืนการเชื่อมต่อ Drive
import os
from google.colab import drive
import shutil
from pathlib import Path

# 1. บังคับตัดการเชื่อมต่อเก่าทิ้ง (เพื่อความชัวร์)
print("🔌 กำลังรีเซ็ตการเชื่อมต่อ Drive...")
try:
    drive.flush_and_unmount()
except:
    pass

# 2. เชื่อมต่อใหม่แบบ Force (จะเด้งหน้าต่าง Login อีกครั้ง)
print("🔑 กรุณา Login Google Drive อีกครั้ง (ต้องเป็นบัญชีเดียวกับที่คุณเช็คอยู่นะ!)")
drive.mount('/content/drive', force_remount=True)

# 3. กำหนดเป้าหมาย
target_dir = Path('/content/drive/MyDrive/8888/ColabDL')

# 4. ตรวจสอบสถานะจริง
print(f"\n📂 ตรวจสอบเส้นทาง: {target_dir}")

if target_dir.exists():
    files = list(target_dir.glob('*'))
    file_count = len(files)

    if file_count > 0:
        print(f"✅ เจอแล้ว!! มีไฟล์ทั้งหมด {file_count} ไฟล์")
        print("--- ตัวอย่างไฟล์ ---")
        for f in files[:5]: # โชว์ 5 ไฟล์แรก
            print(f"📄 {f.name}")
        print("--------------------")
        print("💡 คำแนะนำ: ตอนนี้ไฟล์เชื่อมกับ Cloud แล้ว ให้กลับไปดูที่หน้าเว็บ Google Drive ได้เลย (อาจต้องรอ 1-2 นาที)")
    else:
        print("❌ โฟลเดอร์มีอยู่จริง แต่ 'ว่างเปล่า' (Empty)")
        print("⚠️ วิเคราะห์: แสดงว่ารอบที่แล้ว Copy ไปลง 'โฟลเดอร์หลอก' ในเครื่องจำลอง ไม่ได้ลง Drive จริง")
else:
    print("❌ ไม่เจอโฟลเดอร์นี้เลย")
    print("⚠️ วิเคราะห์: Path อาจจะผิด หรือ โฟลเดอร์ยังไม่ถูกสร้าง")

In [ ]:
# @title 🚑 Ghost File Rescuer & Sync (กู้ไฟล์ที่ติดค้าง)
import os
import shutil
from google.colab import drive
from pathlib import Path

# --- 1. เตรียมพื้นที่พักไฟล์ชั่วคราว ---
fake_drive_path = Path('/content/drive')
rescue_path = Path('/content/TEMP_RESCUE_ZONE')
real_target_path = Path('/content/drive/MyDrive/8888/ColabDL')

print("🕵️‍♂️ กำลังตรวจสอบ Ghost Files...")

# ตรวจสอบว่ามีไฟล์ค้างจริงไหม
if fake_drive_path.exists() and not os.path.ismount('/content/drive'):
    # สร้างโฟลเดอร์พักไฟล์
    if rescue_path.exists():
        shutil.rmtree(rescue_path)
    rescue_path.mkdir(parents=True, exist_ok=True)

    print("📦 พบไฟล์ตกค้าง! กำลังย้ายไปพื้นที่ปลอดภัย...")

    # ค้นหาและย้ายไฟล์ MP4/TS ทั้งหมดออกมา
    moved_count = 0
    for file_path in fake_drive_path.rglob('*'):
        if file_path.is_file() and file_path.suffix.lower() in ['.mp4', '.ts']:
            try:
                shutil.move(str(file_path), str(rescue_path / file_path.name))
                moved_count += 1
                print(f"  - กู้คืน: {file_path.name}")
            except Exception as e:
                print(f"  - ย้ายไม่ได้: {file_path.name} ({e})")

    print(f"✅ ย้ายไฟล์ออกมาพักไว้แล้ว {moved_count} ไฟล์")

    # --- 2. เคลียร์ทาง (ลบโฟลเดอร์ปลอม) ---
    print("🧹 กำลังเคลียร์พื้นที่ Mountpoint...")
    shutil.rmtree(fake_drive_path)
    os.makedirs(fake_drive_path, exist_ok=True) # สร้างโฟลเดอร์เปล่ารอไว้

else:
    print("ℹ️ ไม่พบ Ghost Files ที่ขวางทางอยู่ (หรือ Drive อาจจะ Mount อยู่แล้ว)")

# --- 3. เชื่อมต่อ Drive ของจริง ---
print("\n🔑 กำลังเชื่อมต่อ Google Drive (ของจริง)...")
try:
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print(f"❌ Mount ไม่สำเร็จ: {e}")
    print("คำแนะนำ: ให้ Factory Reset Runtime แล้วเริ่มใหม่แต่ต้น")
    raise e

# --- 4. ส่งไฟล์กลับเข้า Cloud ---
if rescue_path.exists() and any(rescue_path.iterdir()):
    print(f"\n🚀 กำลังอัปโหลดไฟล์จากที่พัก เข้าสู่ Google Drive...")

    # ตรวจสอบว่าโฟลเดอร์ปลายทางใน Drive มีอยู่จริงไหม
    real_target_path.mkdir(parents=True, exist_ok=True)

    files_to_move = list(rescue_path.glob('*'))
    success_count = 0

    for f in files_to_move:
        dest = real_target_path / f.name
        try:
            if not dest.exists(): # ป้องกันการทับไฟล์เดิมที่มีอยู่แล้ว
                shutil.move(str(f), str(dest))
                print(f"  - Uploaded: {f.name}")
                success_count += 1
            else:
                print(f"  - Skipped (ไฟล์ซ้ำ): {f.name}")
        except Exception as e:
            print(f"  ❌ Failed: {f.name} - {e}")

    print(f"\n🎉 กู้คืนและอัปโหลดสำเร็จ: {success_count} ไฟล์")
    print("👉 ไปเช็คที่หน้าเว็บ Google Drive ได้เลยครับ!")

    # ลบโฟลเดอร์พักไฟล์
    shutil.rmtree(rescue_path)

else:
    print("\n✅ Drive เชื่อมต่อแล้ว และไม่มีไฟล์ตกค้างให้กู้คืนครับ")